# Facial Recognition Project

## Imports

In [1]:
# Import standard dependicies
import cv2
import os
import uuid
from matplotlib import pyplot as plt
import random
import numpy as np

# Import tensorflow dependicies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

## Constants

In [2]:
# Setup paths
ANC_PATH = os.path.join('..', 'data', 'anchor')
POS_PATH = os.path.join('..', 'data', 'positive')
NEG_PATH = os.path.join('..', 'data', ' negative')

## Functions

In [3]:
def calc_cropping_boundaries(frame_dim, target_dim) -> int:
    """
    Calculate the pixel boundaries (width, height) for cropping the webcam frame.
    """
    frame_dim_centre = int(frame_dim / 2)
    half_target_dim = int(target_dim / 2)
    
    starting_pixel = frame_dim_centre - half_target_dim
    ending_pixel = frame_dim_centre + half_target_dim
    
    return starting_pixel, ending_pixel

## Allow user to take pictures of their face (using laptop webcam)

In [4]:
def webcam(width_pixels, height_pixels, key):
    # Establish connection to webcam
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        val, frame = cap.read()

        # Identify width and height of webcam frame
        frame_width = frame.shape[0]
        frame_height = frame.shape[1]

        # Calculate boundaries for central cropping
        start_width, end_width = calc_cropping_boundaries(frame_width, width_pixels)
        start_height, end_height = calc_cropping_boundaries(frame_height, height_pixels)

        # Crop frame so that captured images are same size as labelled images (dimensions 250 by 250 pixels)
        cropped_frame = frame[start_width:end_width, start_height:end_height, :]
        
        if key == 'a':
            # Capture anchor image
            if cv2.waitKey(1) & 0XFF == ord('a'):
                # Create unique file path for anchor image
                img_path = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))

                # Write anchor image to unique file path
                cv2.imwrite(img_path, cropped_frame)
                
        if key == 'p':
            # Capture positive image
            if cv2.waitKey(1) & 0XFF == ord('p'):
                # Create unique file path for positive image
                img_path = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))

                # Write anchor image to unique file path
                cv2.imwrite(img_path, cropped_frame)

        # Show image back to screen
        cv2.imshow('Image cap', cropped_frame)

        # Wait for a key press, and break loop if 'q' key is held
        if cv2.waitKey(1) & 0XFF == ord('q'):
            break

    # Release the webcam
    cap.release()

    # Close image show frame
    cv2.destroyAllWindows()

In [5]:
# Define desired size of image [width, height] so that frame can be cropped
width_pixels, height_pixels = [250, 250]

In [6]:
# Take Anchor images with webcam
webcam(width_pixels, height_pixels, 'a')

In [33]:
# Take Positive images with webcam
webcam(width_pixels, height_pixels, 'p')

## Load and preprocess images

In [7]:
# Pull image paths in a streaming fashion
anchor = tf.data.Dataset.list_files(ANC_PATH+'/*.jpg').take(400)
positive = tf.data.Dataset.list_files(POS_PATH+'/*.jpg').take(400)
negative = tf.data.Dataset.list_files(NEG_PATH+'/*.jpg').take(400)

In [8]:
# # Access the image paths from within the tensors
# dir_test = anchor.as_numpy_iterator()

# # Iterate through the image paths one at a time
# dir_test.next()

In [71]:
def read_img(file_path):
    # Read image from file path
    byte_img = tf.io.read_file(file_path)
    
    # Decode image
    img = tf.io.decode_jpeg(byte_img)
    
    return img


def preprocess(file_path):
    
    img = read_img(file_path)
    
    # Resize the image to be 105 x 105 x 3 (chose 105 as per Siamese Neural Network research paper)
    img = tf.image.resize(img, (105, 105))
    
    # Scale the pixels to be values between 0 and 1
    img = img / 255
    
    return img

## Create labelled dataset

In [137]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

## Build and train test partition

In [138]:
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [139]:
# Dataloader pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [141]:
# Training partition
train_data = data.take(round(len(data) * 0.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [142]:
# Testing partition
test_data = data.skip(round(len(data) * 0.7))
test_data = data.take(round(len(data) * 0.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

## Build embedding layer

In [154]:
def make_embedding(): 
    input_layer = Input(shape=(105,105,3), name='input_image')
    
    # First block
    c1 = Conv2D(64, (10,10), activation='relu')(input_layer)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)
    
    # Second block
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)
    
    # Third block 
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
    
    # Final embedding block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    
    return Model(inputs=[input_layer], outputs=[d1], name='embedding')

In [155]:
embedding = make_embedding()
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 105, 105, 3)]     0         
                                                                 
 conv2d_5 (Conv2D)           (None, 96, 96, 64)        19264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 64)       0         
 )                                                               
                                                                 
 conv2d_6 (Conv2D)           (None, 42, 42, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 21, 21, 128)      0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 18, 18, 128)       26

## Build distance layer

In [159]:
class L1Dist(Layer):
    
    def __init__(self, **kwargs):
        super().__init__()
    
    # Calculate similarity (distance)
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

## Make Siamese model

In [182]:
def make_siamese_model():
    
    # Input anchor image 
    input_image = Input(name='input_img', shape=(105,105,3))
    
    # Input validation image
    validation_image = Input(name='validation_img', shape=(105,105,3))
    
    # Combine Siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    # Classification layer
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [ ]:
make_siamese_model